In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

# Fetch and Clean Data


In [24]:
# companies = ["POLYCAB", "ITC", "LTTS", "HCLTECH", "NMDC", "DRREDDY", "DEEPAKNTR"]
companies = [
    "LTTS",
    "GMDCLTD",
    "PIIND",
    "SKFINDIA",
    "DRREDDY",
    "INFY",
    "HCLTECH",
    "CUMMINSIND",
]
N = len(companies)
rf = 0.0721


def get_returns(companies: list[str]):
    companies_str = " ".join([c + ".NS" for c in companies])
    df: pd.DataFrame = yf.download(companies_str, period="1y").dropna()["Adj Close"]
    return df.pct_change().dropna()


def get_latest_price(companies: list[str]):
    companies_str: str = " ".join([c + ".NS" for c in companies])
    df: pd.DataFrame = yf.download(companies_str, period="1D").dropna()["Adj Close"]
    return df


price = get_latest_price(companies)
df = get_returns(companies)
df

[*********************100%%**********************]  8 of 8 completed
[*********************100%%**********************]  8 of 8 completed


,CUMMINSIND.NS,DRREDDY.NS,GMDCLTD.NS,HCLTECH.NS,INFY.NS,LTTS.NS,PIIND.NS,SKFINDIA.NS
Date,,,,,,,,
2022-11-18,-0.022379,-0.002771,-0.007060,0.009702,0.003780,0.013008,-0.021504,0.010697
2022-11-21,-0.020558,-0.013427,0.019908,-0.006935,-0.015815,0.001275,0.008321,0.026155
2022-11-22,0.006013,0.002908,-0.011851,0.007211,0.010745,0.012234,0.002600,-0.014627
2022-11-23,0.017631,0.009891,0.005291,0.000000,-0.001199,-0.008584,0.012338,-0.012240
2022-11-24,0.017769,0.001861,-0.000351,0.023745,0.029467,0.017736,0.003969,0.005153
...,...,...,...,...,...,...,...,...
2023-11-10,0.010254,-0.006055,0.036916,-0.009507,-0.004328,0.005185,0.010828,-0.007461
2023-11-13,0.005655,-0.000931,0.024921,0.004421,0.003434,-0.004058,-0.004515,-0.013681
2023-11-15,0.027159,0.000018,-0.006561,0.012411,0.026901,0.031308,0.012559,0.019790


In [25]:
exp_ret = (((1 + df.mean()) ** 252) - 1).to_numpy()

exp_ret = exp_ret.reshape(len(exp_ret), 1)
exp_ret

array([[ 0.39969812],
       [ 0.30656434],
       [ 2.36123132],
       [ 0.29257947],
       [-0.02622273],
       [ 0.24834128],
       [ 0.15266004],
       [-0.03267353]])

In [26]:
cov_mat = (df.cov() * 252).to_numpy()
cov_mat

array([[ 5.95986250e-02,  7.82343916e-03,  1.53068180e-02,
         4.26309005e-03,  1.48966484e-02,  1.48740975e-02,
         1.35730265e-02,  5.97021991e-03],
       [ 7.82343916e-03,  3.32317473e-02, -4.01698311e-03,
         5.64735731e-03,  7.89555703e-03,  2.72808654e-03,
         9.23863271e-03,  2.22591096e-04],
       [ 1.53068180e-02, -4.01698311e-03,  3.20446041e-01,
         8.20656809e-03, -1.33614537e-03,  9.86747671e-03,
         2.25946914e-02,  1.50922513e-02],
       [ 4.26309005e-03,  5.64735731e-03,  8.20656809e-03,
         4.03050481e-02,  2.77685199e-02,  2.54108948e-02,
         1.13653770e-02,  5.19762465e-03],
       [ 1.48966484e-02,  7.89555703e-03, -1.33614537e-03,
         2.77685199e-02,  5.28560436e-02,  2.86013309e-02,
         9.29270290e-03,  5.79513328e-03],
       [ 1.48740975e-02,  2.72808654e-03,  9.86747671e-03,
         2.54108948e-02,  2.86013309e-02,  7.74692798e-02,
         1.58602675e-02,  1.73905230e-02],
       [ 1.35730265e-02,  9.238632

In [27]:
def sharpe(w: np.ndarray, mu: np.ndarray, sigma: np.ndarray, rf: float):
    return ((w.T @ mu - rf) / (w.T @ sigma @ w) ** 0.5)[0][0]


def init_w(n: int, random=True):
    if random:
        w = np.random.uniform(0, 1, (n, 1))
        w /= w.sum()
        return w
    else:
        w = np.zeros((n, 1))
        w[0][0] = 1
        return w


sharpe(init_w(N), exp_ret, cov_mat, rf)

2.9489308284402784

# Genetic Algorithm


In [28]:
def fitness(w: np.ndarray, tangency=True):
    if abs(w.sum() - 1) > 1e-2:
        return -(1e9 + 7)
    if tangency:
        return sharpe(w, exp_ret, cov_mat, 0.05)
    else:
        return (-w.T @ cov_mat @ w).flatten()[0]

## Hyper-Parameters


In [29]:
n_iter = 2000
mutation_variation = 0.1
offsprings = 200
top_select = offsprings // 2

## Run the Model


In [33]:
solutions = np.array([init_w(N) for _ in range(offsprings)])

for i in range(n_iter):
    fitness_values = np.array([fitness(w, tangency=False) for w in solutions])
    ranked_indices = np.argsort(fitness_values)[::-1]
    solutions = solutions[ranked_indices.astype(int)]

    best = solutions[:top_select]

    selected_indices = np.random.choice(top_select, size=offsprings)
    mutated = best[selected_indices] * np.random.uniform(
        1 - mutation_variation / 2, 1 + mutation_variation / 2, (offsprings, N, 1)
    )
    mutated /= mutated.sum(axis=1, keepdims=True)

    solutions = mutated

for i in range(5):
    print(
        f"optimal solution {i+1}: sharpe ratio = {round(sharpe(solutions[i], exp_ret, cov_mat, rf), 3)} with weights = {np.round(solutions[i].T, 4).flatten().tolist()}"
    )

w = solutions[0]

optimal solution 1: sharpe ratio = 1.538 with weights = [0.1137, 0.3287, 0.0195, 0.1958, 0.0426, 0.0004, 0.0774, 0.2221]
optimal solution 2: sharpe ratio = 1.597 with weights = [0.1211, 0.3111, 0.022, 0.2099, 0.0288, 0.0004, 0.0724, 0.2343]
optimal solution 3: sharpe ratio = 1.582 with weights = [0.1232, 0.3318, 0.0198, 0.193, 0.0414, 0.0004, 0.077, 0.2133]
optimal solution 4: sharpe ratio = 1.561 with weights = [0.1185, 0.3114, 0.0202, 0.2088, 0.0293, 0.0006, 0.0806, 0.2305]
optimal solution 5: sharpe ratio = 1.563 with weights = [0.1177, 0.3251, 0.0205, 0.1972, 0.0364, 0.0004, 0.0747, 0.228]


# Overall Portfolio


In [34]:
capital = 100000

print(f"For a capital amount of Rs. {capital}:")
for i in range(N):
    print(f"Invest Rs. {round(w[i][0] * capital, 2)} in {companies[i]}")

print(
    f"\nExpected Return of Entire Portfolio: {round((w.T @ exp_ret)[0][0] * 100, 2)}%"
)
print(f"Risk of Entire Portfolio: {round((w.T @ cov_mat @ w)[0][0] * 100, 2)}%")

For a capital amount of Rs. 100000:
Invest Rs. 11368.84 in LTTS
Invest Rs. 32865.22 in GMDCLTD
Invest Rs. 1945.87 in PIIND
Invest Rs. 19578.54 in SKFINDIA
Invest Rs. 4262.06 in DRREDDY
Invest Rs. 38.87 in INFY
Invest Rs. 7735.15 in HCLTECH
Invest Rs. 22205.45 in CUMMINSIND

Expected Return of Entire Portfolio: 25.3%
Risk of Entire Portfolio: 1.38%


In [35]:
w_eq = np.ones((N, 1)) / N
w_eq.T @ exp_ret
w_eq.T @ cov_mat @ w

array([[0.01379017]])